In [75]:
import numpy as np
import matplotlib.pyplot as plt
import zipfile
try:
  import google.colab
except ImportError:
  from dolfin import *
else:
  try:
    from dolfin import *
  except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    from dolfin import *
%matplotlib inline

In [ ]:
# 1. Read the mesh for this problem

!wget "https://github.com/niravshah241/transient_heat_conduction_LSTM_PINN/archive/refs/heads/main.zip" -O "/tmp/github_files.zip"

zip_ref = zipfile.ZipFile("/tmp/github_files.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

mesh = Mesh("/tmp/transient_heat_conduction_LSTM_PINN-main/mesh_data/mesh.xml")
subdomains = MeshFunction("size_t", mesh, "/tmp/transient_heat_conduction_LSTM_PINN-main/mesh_data/mesh_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "/tmp/transient_heat_conduction_LSTM_PINN-main/mesh_data/mesh_facet_region.xml")

dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)
ds_sym = ds(2) + ds(3) + ds(6)
ds_bottom = ds(1)
ds_sf = ds(5) + ds(7) + ds(8) + ds(9) + ds(10)
ds_top = ds(4) + ds(11) + ds(12) + ds(13) + ds(14)
ds_out = ds(15)
dx_cc1 = dx(1) 
dx_cc2 = dx(2)
dx_cc2_upper = dx(4)
dx_bb_surround = dx(3)
dx_cc = dx(6)
dx_cb = dx(5)
dx_ss = dx(7)

# Time scale
t_max = 2.
num_steps = 100
dt = t_max/num_steps

In [77]:
# 2A. Create Finite Element space (Lagrange P1)
VT = FunctionSpace(mesh,"CG",1)
T_, psi = TrialFunction(VT), TestFunction(VT)
T = Function(VT,name="Temperature")
T_previous = interpolate(Constant(300.),VT)

if VT.ufl_element().degree() == 1:
  VQ = VectorFunctionSpace(mesh,"CG",VT.ufl_element().degree()) # Space for heat flux
else:
  VQ = VectorFunctionSpace(mesh,"CG",VT.ufl_element().degree()-1) # Space for heat flux
q_, psi_q = TrialFunction(VQ), TestFunction(VQ)
q = Function(VQ,name="Heat flux") # Heat flux at current time step

#x0 = Expression("x[0]",element=VT.ufl_element())
#x1 = Expression("x[1]",element=VT.ufl_element())

In [78]:
# 3. Material and boundary properties, Source term

k_cc1, k_cc2, k_cc2_upper, k_bb_surround, k_cb, k_ss, k_cc = 10., 10., 10., 10., 10., 10., 10.#15.5, 39.5, 39.5, 20.2, 5.5, 48., 6. #as_tensor([[7.,0],[0,5.]]) # Thermal conductivities 
rho_cc1, rho_cc2, rho_cc2_upper, rho_bb_surround, rho_cb, rho_ss, rho_cc = 1., 1., 1., 1., 1., 1., 1.#3520.0e-2, 3200.0e-2, 3200.0e-2, 2800.0e-2, 1760.0e-2, 7900.0e-2, 2080.0e-2 # Material densities
heat_capacity_cc1, heat_capacity_cc2, heat_capacity_cc2_upper, heat_capacity_bb_surround, heat_capacity_cb, heat_capacity_ss, heat_capacity_cc \
= 1., 1., 1., 1., 1., 1., 1. #744.2e-2, 683.4e-2, 683.4e-2, 705.9e-2, 342.7e-2, 466.0e-2, 342.7e-2 # Heat capacities
h_sf, h_out, h_bottom = 100., 50., 50. # Convection coefficients
T_left = Expression("300.+600.*t", degree=2, t=0.) # Temperature on left boundary at current time step
T_left_previous = interpolate(T_left,VT) # Temperature on left boundary at previous time step
T_right = Expression("300.+0*t",degree=2,t=0) # Temperature on right boundary at current time step
T_right_previous = Expression("300.+0*t",degree=2,t=0) # Temperature on right boundary at previous time step
T_bottom = Expression("300.+0*t",degree=2,t=0) # Temperature on bottom boundary at current time step
T_bottom_previous = Expression("300.+0*t",degree=2,t=0) # Temperature on bottom boundary at previous time step
Q = Expression("0*t", degree=2, t=0.) # Source term at current time step
Q_previous = interpolate(Q,VT) # Source term at previous time step

In [79]:
# 4. Compute heat flux

def compute_heat_flux(temp_field):
  '''
  Compute the heat flux at given temperature field
  Input:
  temp_field: Function over Functionspace
  Output:
  q: Function over VectorFunctionSpace
  '''
  a_q = inner(q_, psi_q) * dx_cc1 + inner(q_, psi_q) * dx_cc2 + inner(q_, psi_q) * dx_cc2_upper + inner(q_, psi_q) * dx_bb_surround + inner(q_, psi_q) * dx_cb \
  + inner(q_, psi_q) * dx_ss + inner(q_, psi_q) * dx_cc
  l_q = inner(-k_cc1 * grad(temp_field), psi_q) * dx_cc1 + inner(-k_cc2 * grad(temp_field), psi_q) * dx_cc2 + inner(-k_cc2_upper * grad(temp_field), psi_q) * dx_cc2_upper + \
  inner(-k_bb_surround * grad(temp_field), psi_q) * dx_bb_surround + inner(-k_cb * grad(temp_field), psi_q) * dx_cb + inner(-k_ss * grad(temp_field), psi_q) * dx_ss + \
  inner(-k_cc * grad(temp_field), psi_q) * dx_cc
  solve(a_q == l_q, q)
  return q

In [ ]:
# 5. Weak formulation with generalised-$\theta$ method in time

theta = Constant(1.) # theta=0 means forward difference and theta=1 means backward difference

F_T = \
psi * rho_cc1 * heat_capacity_cc1 * T_ * dx_cc1 + psi * rho_cc2 * heat_capacity_cc2 * T_ * dx_cc2 + psi * rho_cc2_upper * heat_capacity_cc2_upper * T_ * dx_cc2_upper + \
psi * rho_bb_surround * heat_capacity_bb_surround * T_ * dx_bb_surround + psi * rho_cb * heat_capacity_cb * T_ * dx_cb + psi * rho_ss * heat_capacity_ss * T_ * dx_ss + \
psi * rho_cc * heat_capacity_cc * T_ * dx_cc - \
psi * rho_cc1 * heat_capacity_cc1 * T_previous * dx_cc1 - psi * rho_cc2 * heat_capacity_cc2 * T_previous * dx_cc2 - psi * rho_cc2_upper * heat_capacity_cc2_upper * T_previous * dx_cc2_upper - \
psi * rho_bb_surround * heat_capacity_bb_surround * T_previous * dx_bb_surround - psi * rho_cb * heat_capacity_cb * T_previous * dx_cb - \
psi * rho_ss * heat_capacity_ss * T_previous * dx_ss - psi * rho_cc * heat_capacity_cc * T_previous * dx_cc

F_T_backward = \
dt * inner(k_cc1 * grad(T_), grad(psi)) * dx_cc1 + dt * inner(k_cc2 * grad(T_), grad(psi)) * dx_cc2 + dt * inner(k_cc2_upper * grad(T_), grad(psi)) * dx_cc2_upper + \
dt * inner(k_bb_surround * grad(T_), grad(psi)) * dx_bb_surround + dt * inner(k_cb * grad(T_), grad(psi)) * dx_cb + dt * inner(k_ss * grad(T_), grad(psi)) * dx_ss + \
dt * inner(k_cc * grad(T_), grad(psi)) * dx_cc + \
dt * psi * h_sf * (T_ - T_left) * ds_sf + dt * psi * h_out * (T_ - T_right) * ds_out + dt * psi * h_bottom * (T_ - T_bottom) * ds_bottom - \
psi * dt * Q * dx

F_T_forward = \
dt * inner(k_cc1 * grad(T_previous), grad(psi)) * dx_cc1 + dt * inner(k_cc2 * grad(T_previous), grad(psi)) * dx_cc2 + dt * inner(k_cc2_upper * grad(T_previous), grad(psi)) * dx_cc2_upper + \
dt * inner(k_bb_surround * grad(T_previous), grad(psi)) * dx_bb_surround + dt * inner(k_cb * grad(T_previous), grad(psi)) * dx_cb + dt * inner(k_ss * grad(T_previous), grad(psi)) * dx_ss + \
dt * inner(k_cc * grad(T_previous), grad(psi)) * dx_cc + \
dt * psi * h_sf * (T_previous - T_left_previous) * ds_sf + dt * psi * h_out * (T_previous - T_right_previous) * ds_out + dt * psi * h_bottom * (T_previous - T_bottom_previous) * ds_bottom - \
psi * dt * Q_previous * dx

a_T = lhs(F_T + theta * F_T_backward + (1 - theta) * F_T_forward)
l_T = rhs(F_T + theta * F_T_backward + (1 - theta) * F_T_forward)

#Solution of system of equations
t = 0.
xdmf_file = XDMFFile("/tmp/transient_heat_conduction_LSTM_PINN-main/solution_field/thermal_results.xdmf") #Path to save output files in google drive
xdmf_file.parameters["flush_output"] = True
xdmf_file.parameters["functions_share_mesh"] = True
xdmf_file.parameters["rewrite_function_mesh"] = False
xdmf_file.write(T_previous,0)
xdmf_file.write(compute_heat_flux(T_previous),0)

for i in range(num_steps):
  T_left_previous.t = t
  T_right_previous.t = t
  T_bottom_previous.t = t
  Q_previous.t = t
  t += dt
  T_left.t = t
  T_right.t = t
  T_bottom.t = t
  Q.t = t
  solve(a_T == l_T, T)
  q = compute_heat_flux(T)
  xdmf_file.write(T,t)
  xdmf_file.write(q,t)
  T_previous.assign(T)

plot(T)
plt.show()